In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, mean_squared_error

In [4]:
base_dir = "/Users/fabiancordenod/code/fqbq69/BIMpredict-/raw_data"

dfs = []
for i in range(1, 7):
    poutres_path = os.path.join(base_dir, f"maquette{i}", f"poutres{i}.csv")
    if os.path.exists(poutres_path):
        with open(poutres_path, encoding="utf-8") as f:
            for idx, line in enumerate(f):
                if line.startswith("Id;"):
                    header_row = idx
                    break
        try:
            df = pd.read_csv(poutres_path, sep=';', header=header_row)
            dfs.append(df)
            print(f"Chargé : {poutres_path} ({df.shape[0]} lignes, {df.shape[1]} colonnes)")
            print(df.head())
        except Exception as e:
            print(f"Erreur de parsing : {poutres_path} -> {e}")
    else:
        print(f"Fichier non trouvé : {poutres_path}")

if dfs:
    poutres_concat = pd.concat(dfs, ignore_index=True)
    print(f"Total concaténé : {poutres_concat.shape[0]} lignes, {poutres_concat.shape[1]} colonnes")
else:
    poutres_concat = pd.DataFrame()
    print("Aucun fichier murs.csv trouvé.")

poutres_concat.head()

Chargé : /Users/fabiancordenod/code/fqbq69/BIMpredict-/raw_data/maquette1/poutres1.csv (778 lignes, 136 colonnes)
       Id 011EC_Lot 012EC_Ouvrage 013EC_Localisation 014EC_Mode Constructif  \
0  822465        GO        POUTRE          INTERIEUR         PREFA CHANTIER   
1  822610        GO        POUTRE          INTERIEUR         PREFA CHANTIER   
2  823043        GO   BANDE NOYEE            COURANT         COULE EN PLACE   
3  823110        GO   BANDE NOYEE            COURANT         COULE EN PLACE   
4  823289        GO   BANDE NOYEE            COURANT         COULE EN PLACE   

       Nom                  AI                 AS Hauteur totale  \
0  30x60ht  -0,200000000000001  0,200000000000001            0,6   
1  25x50ht  -0,150000000000001  0,150000000000001            0,5   
2  30x20ht                -0,1                0,1            0,2   
3  30x20ht                -0,1                0,1            0,2   
4  30x20ht                -0,1                0,1            0,2   

  

,Id,011EC_Lot,012EC_Ouvrage,013EC_Localisation,014EC_Mode Constructif,Nom,AI,AS,Hauteur totale,Hauteur,...,Nature_Ouvrage,Batiment,Perimetre coffrage,Vtotal,hauteur_section,largeur_section,NIVEAU_STRUCTURE,Type Ossature,LG_DECAISSE,ht-decaisse
0,822465,GO,POUTRE,INTERIEUR,PREFA CHANTIER,30x60ht,"-0,200000000000001","0,200000000000001","0,6","0,400000000000002",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,822610,GO,POUTRE,INTERIEUR,PREFA CHANTIER,25x50ht,"-0,150000000000001","0,150000000000001","0,5","0,300000000000002",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,823043,GO,BANDE NOYEE,COURANT,COULE EN PLACE,30x20ht,"-0,1","0,1","0,2","0,2",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,823110,GO,BANDE NOYEE,COURANT,COULE EN PLACE,30x20ht,"-0,1","0,1","0,2","0,2",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,823289,GO,BANDE NOYEE,COURANT,COULE EN PLACE,30x20ht,"-0,1","0,1","0,2","0,2",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
poutres_concat.shape

(1846, 158)

In [6]:
print(poutres_concat)

          Id 011EC_Lot 012EC_Ouvrage 013EC_Localisation  \
0     822465        GO        POUTRE          INTERIEUR   
1     822610        GO        POUTRE          INTERIEUR   
2     823043        GO   BANDE NOYEE            COURANT   
3     823110        GO   BANDE NOYEE            COURANT   
4     823289        GO   BANDE NOYEE            COURANT   
...      ...       ...           ...                ...   
1841  380511        GO      LONGRINE          EXTERIEUR   
1842  380519        GO      LONGRINE          EXTERIEUR   
1843  385454        GO      LONGRINE          EXTERIEUR   
1844  385862        GO      LONGRINE          EXTERIEUR   
1845  385870        GO      LONGRINE          EXTERIEUR   

     014EC_Mode Constructif        Nom                  AI                 AS  \
0            PREFA CHANTIER    30x60ht  -0,200000000000001  0,200000000000001   
1            PREFA CHANTIER    25x50ht  -0,150000000000001  0,150000000000001   
2            COULE EN PLACE    30x20ht          

In [9]:
colonnes_a_garder = [
    "011EC_Lot",
    "012EC_Ouvrage",
    "013EC_Localisation",
    "014EC_Mode Constructif",
    "Epaisseur",
    "Sols en intersection",
    "Sols coupés (u)",
    "Sols coupants (u)",
    "Sol au-dessus",
    "Sol en-dessous",
    "Fenêtres",
    "Portes",
    "Ouvertures",
    "Murs imbriqués",
    "Mur multicouche",
    "Profil modifié",
    "Extension inférieure",
    "Extension supérieure",
    "Partie inférieure attachée",
    "Partie supérieure attachée",
    "Décalage supérieur",
    "Décalage inférieur",
    "Matériau structurel"
]

# On ne garde que les colonnes présentes dans le DataFrame
poutres_concat = poutres_concat[[col for col in colonnes_a_garder if col in poutres_concat.columns]]

In [10]:
print(poutres_concat.columns.tolist())

['011EC_Lot', '012EC_Ouvrage', '013EC_Localisation', '014EC_Mode Constructif', 'Sols en intersection', 'Sols coupés (u)', 'Sols coupants (u)', 'Sol au-dessus', 'Sol en-dessous', 'Matériau structurel']


In [11]:
poutres_concat

,011EC_Lot,012EC_Ouvrage,013EC_Localisation,014EC_Mode Constructif,Sols en intersection,Sols coupés (u),Sols coupants (u),Sol au-dessus,Sol en-dessous,Matériau structurel
0,GO,POUTRE,INTERIEUR,PREFA CHANTIER,0,0,1,False,False,Béton - Béton précoulé
1,GO,POUTRE,INTERIEUR,PREFA CHANTIER,0,0,1,False,False,Béton - Béton précoulé
2,GO,BANDE NOYEE,COURANT,COULE EN PLACE,0,1,0,False,False,Béton - Béton précoulé
3,GO,BANDE NOYEE,COURANT,COULE EN PLACE,0,1,0,False,False,Béton - Béton précoulé
4,GO,BANDE NOYEE,COURANT,COULE EN PLACE,0,1,0,False,False,Béton - Béton précoulé
...,...,...,...,...,...,...,...,...,...,...
1841,GO,LONGRINE,EXTERIEUR,COFFRE,0,0,0,False,False,"Concrete, Cast-in-Place gray"
1842,GO,LONGRINE,EXTERIEUR,COFFRE,0,0,0,False,False,"Concrete, Cast-in-Place gray"
1843,GO,LONGRINE,EXTERIEUR,COFFRE,0,0,0,False,False,"Concrete, Cast-in-Place gray"
1844,GO,LONGRINE,EXTERIEUR,COFFRE,0,0,0,False,False,"Concrete, Cast-in-Place gray"


In [13]:
import unicodedata
import re

def clean_col(col):
    # Enlever accents
    col = ''.join(c for c in unicodedata.normalize('NFD', col) if unicodedata.category(c) != 'Mn')
    col = col.lower()
    # Remplacer espaces et tirets par _
    col = re.sub(r"[ \-\(\)]", "_", col)
    # Supprimer tout caractère non alphanumérique ou _
    col = re.sub(r"[^a-z0-9_]", "", col)
    # Nettoyer les doubles/triples underscores
    col = re.sub(r"_+", "_", col)
    # Supprimer _ en début/fin
    col = col.strip("_")
    return col

poutres_concat.columns = [clean_col(c) for c in poutres_concat.columns]
print(poutres_concat.columns.tolist())

['011ec_lot', '012ec_ouvrage', '013ec_localisation', '014ec_mode_constructif', 'sols_en_intersection', 'sols_coupes_u', 'sols_coupants_u', 'sol_au_dessus', 'sol_en_dessous', 'materiau_structurel']


In [14]:
# Définir les targets multi-label (adapte la liste selon tes besoins)
targets = [
    "011ec_lot",
    "012ec_ouvrage",
    "013ec_localisation",
    "014ec_mode_constructif"
]

# Garder seulement les targets présents dans le DataFrame
targets_in_df = [col for col in targets if col in poutres_concat.columns]

if not targets_in_df:
    raise ValueError(f"Aucune colonne cible trouvée dans murs_concat. Colonnes disponibles : {poutres_concat.columns.tolist()}")

# X et y_multi
X = poutres_concat.drop(columns=targets_in_df)
y_multi = poutres_concat[targets_in_df]

if X.shape[1] == 0:
    raise ValueError("Aucune variable explicative disponible après suppression des cibles. Vérifiez vos colonnes.")

num_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_features = X.select_dtypes(include=['object', 'category']).columns.tolist()

# Pipeline de prétraitement
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ]), num_features),
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(handle_unknown='ignore'))
        ]), cat_features)
    ]
)

# Pipeline complet avec MultiOutputClassifier
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', MultiOutputClassifier(RandomForestClassifier(n_estimators=1000, random_state=42)))
])

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y_multi, test_size=0.2, random_state=42)

# Supprimer les lignes avec NaN dans les targets (train et test)
train_notna = y_train.notna().all(axis=1)
test_notna = y_test.notna().all(axis=1)
X_train, y_train = X_train[train_notna], y_train[train_notna]
X_test, y_test = X_test[test_notna], y_test[test_notna]

# Entraînement
pipeline.fit(X_train, y_train)

# Prédiction et score baseline
y_pred = pipeline.predict(X_test)
print("Accuracy moyenne multi-label :", (y_pred == y_test.values).mean())

Accuracy moyenne multi-label : 0.784741144414169


In [ ]:
print("Accuracy calculée sur", len(y_test), "échantillons.")